In [51]:
import requests
from bs4 import BeautifulSoup
import time
import urllib.parse
import numpy as np
import pandas as pd

In [35]:
data_samples = []
max_page = 2
# url = 'https://suumo.jp/jj/chintai/ichiran/FR301FC005/?ar=030&bs=040&pc=30&smk=&po1=25&po2=99&shkr1=03&shkr2=03&shkr3=03&shkr4=03&rn=0305&ek=030506640&ek=030511640&ek=030536880&ek=030538710&ek=030514690&ek=030523100&ra=013&cb=0.0&ct=25.0&co=1&md=07&md=08&md=09&md=10&md=11&md=12&md=13&md=14&et=15&mb=55&mt=9999999&cn=30&fw2='
url = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?fw2=&ek=030506640&ek=030511640&ek=030536880&ek=030538710&ek=030514690&ek=030523100&mt=9999999&cn=30&co=1&ra=013&et=15&smk=&shkr1=03&ar=030&bs=040&ct=25.0&shkr3=03&shkr2=03&mb=55&md=07&md=08&md=09&md=10&md=11&md=12&md=13&md=14&rn=0305&shkr4=03&cb=0.0'
def load_page(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser')
    
    return soup

In [27]:
soup = load_page(url)
main_container = soup.find('div', id='js-bukkenList')

rooms = main_container.find_all('div', class_='property_group')     # All rooms in this page

for room in rooms:
    # Room Name
    header = room.find('div', class_='property-header-inner')
    #print('Room Name:', header.get_text(strip=True))

    # Room Details Top 
    details = room.find('div', class_='detailbox-property')
    rent = details.find('div', class_="detailbox-property-point").get_text()
    print(rent)

    # Room Details Bottom (Distance from station)
    # dist = room.find('div', class_='detailbox-note')
    #print(dist.get_text(strip=True))




    # Find all <div> elements inside this <td>
    divs = details.find_all('div')

    # Extract the prices
    for div in divs:
        # Extract the text from each <div> and remove the label (敷, 礼)
        if div.find('span', class_='square_pct'):  # Check if <span class="square_pct"> is present
            price = div.get_text(strip=True).replace(div.find('span').get_text(strip=True), '').strip()
            print(price)

    break


15.4万円
15.4万円
30.8万円


In [36]:
start = time.time()
times = []

# ページごとの処理
for page in range(1,max_page+1):
    before = time.time()
    # ページ情報
    soup = load_page(url.format(page))
    # 物件情報リストを指定
    mother = soup.find_all(class_='cassetteitem')
    
    # 物件ごとの処理
    for child in mother:

        # 建物情報
        data_home = []
        # カテゴリ
        data_home.append(child.find(class_='ui-pct ui-pct--util1').text)
        # 建物名
        data_home.append(child.find(class_='cassetteitem_content-title').text)
        # 住所
        data_home.append(child.find(class_='cassetteitem_detail-col1').text)
        # 最寄り駅のアクセス
        children = child.find(class_='cassetteitem_detail-col2')
        for id,grandchild in enumerate(children.find_all(class_='cassetteitem_detail-text')):
            data_home.append(grandchild.text)
        # 築年数と階数
        children = child.find(class_='cassetteitem_detail-col3')
        for grandchild in children.find_all('div'):
            data_home.append(grandchild.text)

        # 部屋情報
        rooms = child.find(class_='cassetteitem_other')
        for room in rooms.find_all(class_='js-cassette_link'):
            data_room = []
            
            # 部屋情報が入っている表を探索
            for id_, grandchild in enumerate(room.find_all('td')):
                # 階
                if id_ == 2:
                    data_room.append(grandchild.text.strip())
                # 家賃と管理費
                elif id_ == 3:
                    data_room.append(grandchild.find(class_='cassetteitem_other-emphasis ui-text--bold').text)
                    data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--administration').text)
                # 敷金と礼金
                elif id_ == 4:
                    data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--deposit').text)
                    data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--gratuity').text)
                # 間取りと面積
                elif id_ == 5:
                    data_room.append(grandchild.find(class_='cassetteitem_madori').text)
                    data_room.append(grandchild.find(class_='cassetteitem_menseki').text)
                # url
                elif id_ == 8:
                    get_url = grandchild.find(class_='js-cassette_link_href cassetteitem_other-linktext').get('href')
                    abs_url = urllib.parse.urljoin(url,get_url)
                    data_room.append(abs_url)
            # 物件情報と部屋情報をくっつける
            data_sample = data_home + data_room
            data_samples.append(data_sample)
    
    # 1アクセスごとに1秒休む
    time.sleep(1)
    
    # 進捗確認
    # このページの作業時間を表示
    after = time.time()
    running_time = after - before
    times.append(running_time)
    print(f'{page}ページ目：{running_time}秒')
    # 取得した件数
    print(f'総取得件数：{len(data_samples)}')
    # 作業進捗
    complete_ratio = round(page/max_page*100,3)
    print(f'完了：{complete_ratio}%')
    # 作業の残り時間目安を表示
    running_mean = np.mean(times)
    running_required_time = running_mean * (max_page - page)
    hour = int(running_required_time/3600)
    minute = int((running_required_time%3600)/60)
    second = int(running_required_time%60)
    print(f'残り時間：{hour}時間{minute}分{second}秒\n')
    
# 処理時間を測りたい
finish = time.time()
running_all = finish - start
print('総経過時間：',running_all)

1ページ目：1.8308391571044922秒
総取得件数：43
完了：50.0%
残り時間：0時間0分1秒

2ページ目：1.77482008934021秒
総取得件数：86
完了：100.0%
残り時間：0時間0分0秒

総経過時間： 3.6079790592193604


In [52]:
# ページごとの処理
for page in range(1, max_page + 1):
    before = time.time()
    
    # ページ情報
    soup = load_page(url.format(page))
    
    # 物件情報リストを指定
    mother = soup.find_all(class_='cassetteitem')
    
    # 物件ごとの処理
    for child in mother:
        data_home = []
        
        # 建物情報 (first-level)
        data_home.append(child.find(class_='ui-pct ui-pct--util1').text)
        data_home.append(child.find(class_='cassetteitem_content-title').text)
        data_home.append(child.find(class_='cassetteitem_detail-col1').text)
        
        # 最寄り駅のアクセス (second-level)
        children = child.find(class_='cassetteitem_detail-col2')
        data_home.extend([grandchild.text for grandchild in children.find_all(class_='cassetteitem_detail-text')])
        
        # 築年数と階数 (third-level)
        children = child.find(class_='cassetteitem_detail-col3')
        data_home.extend([grandchild.text for grandchild in children.find_all('div')])
        
        # 部屋情報
        rooms = child.find(class_='cassetteitem_other')
        for room in rooms.find_all(class_='js-cassette_link'):
            data_room = []
            
            # 部屋情報が入っている表を探索
            for id_, grandchild in enumerate(room.find_all('td')):
                if id_ == 2:  # 階
                    data_room.append(grandchild.text.strip())
                elif id_ == 3:  # 家賃と管理費
                    data_room.append(grandchild.find(class_='cassetteitem_other-emphasis ui-text--bold').text)
                    data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--administration').text)
                elif id_ == 4:  # 敷金と礼金
                    data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--deposit').text)
                    data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--gratuity').text)
                elif id_ == 5:  # 間取りと面積
                    data_room.append(grandchild.find(class_='cassetteitem_madori').text)
                    data_room.append(grandchild.find(class_='cassetteitem_menseki').text)
                elif id_ == 8:  # URL
                    get_url = grandchild.find(class_='js-cassette_link_href cassetteitem_other-linktext').get('href')
                    abs_url = urllib.parse.urljoin(url, get_url)
                    data_room.append(abs_url)
            
            # 物件情報と部屋情報をくっつける
            data_samples.append(data_home + data_room)
    
    # 1アクセスごとに1秒休む
    time.sleep(1)
    
    # 進捗確認
    after = time.time()
    running_time = after - before
    times.append(running_time)
    print(f'{page}ページ目：{running_time}秒')
    print(f'総取得件数：{len(data_samples)}')
    
    complete_ratio = round(page / max_page * 100, 3)
    print(f'完了：{complete_ratio}%')
    
    running_mean = np.mean(times)
    running_required_time = running_mean * (max_page - page)
    hour = int(running_required_time / 3600)
    minute = int((running_required_time % 3600) / 60)
    second = int(running_required_time % 60)
    print(f'残り時間：{hour}時間{minute}分{second}秒\n')

# 処理時間を測りたい
finish = time.time()
running_all = finish - start
print('総経過時間：', running_all)

1ページ目：1.8681268692016602秒
総取得件数：129
完了：50.0%
残り時間：0時間0分1秒

2ページ目：1.8341708183288574秒
総取得件数：172
完了：100.0%
残り時間：0時間0分0秒

総経過時間： 1156.0516650676727


In [53]:
cols = ['賃貸タイプ', '物件名', '住所', '駅距離1', '駅距離2', '駅距離3', '築年数', '何階建て', '階', '賃料', '管理費', '敷金', '礼金', '間取り', '面積', 'url']
df = pd.DataFrame(data_samples, columns=cols)

In [54]:
df

,賃貸タイプ,物件名,住所,駅距離1,駅距離2,駅距離3,築年数,何階建て,階,賃料,管理費,敷金,礼金,間取り,面積,url
0,賃貸テラス・タウンハウス,ＪＲ中央線 三鷹駅 2階建 築11年,東京都三鷹市下連雀２,ＪＲ中央線/三鷹駅 歩13分,ＪＲ中央線/吉祥寺駅 歩14分,ＪＲ中央線/武蔵境駅 歩28分,築11年,2階建,2階,15.4万円,-,15.4万円,30.8万円,2LDK,62.41m2,https://suumo.jp/chintai/jnc_000097423726/?bc=...
1,賃貸テラス・タウンハウス,ＪＲ中央線 三鷹駅 2階建 築11年,東京都三鷹市下連雀２,ＪＲ中央線/三鷹駅 歩13分,ＪＲ中央線/吉祥寺駅 歩14分,ＪＲ中央線/武蔵境駅 歩28分,築11年,2階建,1-2階,15.4万円,-,15.4万円,30.8万円,2LDK,62.41m2,https://suumo.jp/chintai/jnc_000097405016/?bc=...
2,賃貸マンション,プレジール立川,東京都立川市錦町６,ＪＲ中央線/立川駅 歩15分,多摩都市モノレール/柴崎体育館駅 歩11分,ＪＲ南武線/西国立駅 歩16分,築20年,6階建,2階,13.2万円,10000円,13.2万円,13.2万円,2LDK,63.22m2,https://suumo.jp/chintai/jnc_000096759485/?bc=...
3,賃貸マンション,プレジール立川,東京都立川市錦町６,ＪＲ中央線/立川駅 歩15分,多摩都市モノレール/柴崎体育館駅 歩11分,ＪＲ南武線/西国立駅 歩16分,築20年,6階建,4階,13.3万円,10000円,13.3万円,13.3万円,2SLDK,63.22m2,https://suumo.jp/chintai/jnc_000089398419/?bc=...
4,賃貸マンション,プレジール立川,東京都立川市錦町６,ＪＲ中央線/立川駅 歩15分,多摩都市モノレール/柴崎体育館駅 歩11分,ＪＲ南武線/西国立駅 歩16分,築20年,6階建,1階,14万円,10000円,14万円,14万円,2LDK,63.22m2,https://suumo.jp/chintai/jnc_000097448664/?bc=...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,賃貸アパート,ＪＲ中央線 武蔵小金井駅 2階建 築14年,東京都小金井市前原町２,ＪＲ中央線/武蔵小金井駅 歩14分,ＪＲ中央線/東小金井駅 歩27分,西武多摩川線/新小金井駅 歩26分,築14年,2階建,2階,14.2万円,4500円,-,14.2万円,2LDK,55.45m2,https://suumo.jp/chintai/jnc_000096157276/?bc=...
168,賃貸アパート,ＪＲ中央線 武蔵小金井駅 2階建 築14年,東京都小金井市前原町２,ＪＲ中央線/武蔵小金井駅 歩14分,ＪＲ中央線/東小金井駅 歩27分,西武多摩川線/新小金井駅 歩26分,築14年,2階建,2階,14.2万円,4500円,-,14.2万円,2LDK,55.45m2,https://suumo.jp/chintai/jnc_000096157277/?bc=...
169,賃貸一戸建て,Letoit立川A,東京都立川市柴崎町２,多摩都市モノレール/立川南駅 歩7分,ＪＲ中央線/立川駅 歩9分,ＪＲ青梅線/西立川駅 歩26分,築3年,2階建,2階,24万円,3000円,24万円,24万円,3LDK,83.48m2,https://suumo.jp/chintai/jnc_000096020489/?bc=...
170,賃貸テラス・タウンハウス,フォレ下連雀,東京都三鷹市下連雀２,ＪＲ中央線/三鷹駅 歩13分,ＪＲ中央線/吉祥寺駅 バス12分 (バス停)万助橋 歩7分,京王井の頭線/井の頭公園駅 歩25分,築12年,2階建,1階,15.4万円,-,15.4万円,30.8万円,2LDK,62.41m2,https://suumo.jp/chintai/jnc_000097415156/?bc=...
